In [49]:
import pandas as pd
import numpy as np

In [52]:
df_attacks = pd.read_csv('CICIDSFULLOnlyAttacks.csv')
df_normal = pd.read_csv('CICIDSFULLOnlyNormal.csv')

ModuleNotFoundError: No module named 'google.colab'

In [41]:
df = pd.concat([df_attacks, df_normal])

In [42]:
df.replace({'Benign': 0, 'DDOS attack-HOIC': 1, 'DDoS attacks-LOIC-HTTP': 1, 
            'DoS attacks-Hulk': 1, 'Bot': 1, 'FTP-BruteForce': 1, 
            'SSH-Bruteforce': 1, 'Infilteration': 1, 
            'DoS attacks-SlowHTTPTest': 1, 'DoS attacks-GoldenEye': 1, 'DoS attacks-Slowloris': 1, 
            'DDOS attack-LOIC-UDP': 1, 'Brute Force -Web': 1, 'Brute Force -XSS': 1, 'SQL Injection': 1}, inplace=True)

C:\Users\mikae\AppData\Local\Temp\ipykernel_7420\1506006347.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Benign': 0, 'DDOS attack-HOIC': 1, 'DDoS attacks-LOIC-HTTP': 1,


In [58]:
x = df.drop('label', axis=1)
y = df['label']
y.value_counts()

label
1    2746934
0    2746900
Name: count, dtype: int64

In [63]:
count = np.sum(np.isclose(x.values, 0))
count

np.int64(181965165)

In [62]:
for column in x.columns:
    mediana = x[column].median()
    x[column] = x[column].replace(0, mediana)

In [57]:
x

,dst_port,protocol,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,...,fwd_act_data_pkts,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
0,443,6,198,3,1,77.0,56.0,46.0,0.0,25.666667,...,1,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,443,6,223,3,1,77.0,56.0,46.0,0.0,25.666667,...,1,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,443,6,587,3,1,77.0,56.0,46.0,0.0,25.666667,...,1,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,443,6,763,3,1,77.0,56.0,46.0,0.0,25.666667,...,1,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,443,6,852,3,1,77.0,56.0,46.0,0.0,25.666667,...,1,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746895,53,17,261,1,1,65.0,193.0,65.0,65.0,65.000000,...,0,8,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2746896,443,6,101362706,16,17,1161.0,3774.0,613.0,0.0,72.562500,...,12,20,144393.4,131615.682101,518968.0,102212.0,9991877.2,33020.691115,10006947.0,9906973.0
2746897,3389,6,1637318,8,7,1144.0,1581.0,677.0,0.0,143.000000,...,5,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2746898,3389,6,2244273,8,7,1148.0,1581.0,677.0,0.0,143.500000,...,5,20,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [61]:
count = np.sum(np.isclose(x.values, 0))
count

np.int64(181965165)